In [1]:
import os, openai
os.environ["OPENAI_API_KEY"] = 'INSERT_YOUR_OPENAI_KEY_HERE'
openai.api_key = os.environ["OPENAI_API_KEY"] 


In [2]:
# set text wrapping
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Ingest Unstructured Data Through the Unstructured.io Reader

Leverage the capabilities of Unstructured.io CSV for parsing

In [3]:
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from pathlib import Path

#list ema docs
ema = [1,2,3,4]

UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)
loader = UnstructuredReader()
doc_set = {}
all_docs = []
for ema_num in ema:
    ema_docs = loader.load_data(file=Path(f'./data/EMA/EMA_{ema_num}.csv'), split_documents=False)
    # insert year metadata into each year
    for d in ema_docs:
        d.extra_info = {"ema_num": ema_num}
    doc_set[ema_num] = ema_docs
    all_docs.extend(ema_docs)

/Users/sean/.pyenv/versions/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/sean/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sean/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Setup a Vector Index for each EMA doc in the data file

We setup a separate vector index for each file 

We also optionally initialize a "global" index by dumping all files into the vector store.

In [4]:
# initialize simple vector indices + global vector index
# NOTE: don't run this cell if the indices are already loaded! 
index_set = {}
service_context = ServiceContext.from_defaults(chunk_size=512)
for ema_num in ema:
    cur_index = VectorStoreIndex.from_documents(doc_set[ema_num], service_context=service_context)
    index_set[ema_num] = cur_index

# Load indices from disk
index_set = {}
for ema_num in ema:
    index_set[ema_num] = cur_index
    

### Composing a Graph to synthesize answers across all the existing EMA docs. 

We want our queries to aggregate/synthesize information across *all* docs. To do this, we define a List index
on top of the 4 vector indices.

In [5]:
from llama_index import ListIndex, LLMPredictor
from langchain import OpenAI
from llama_index.indices.composability import ComposableGraph

index_summaries = [f"These are the official documents from EMA. This is document index {ema_num}." for ema_num in ema]

# set number of output tokens
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, max_tokens=512))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# define a list index over the vector indices
# allows us to synthesize information across each index
graph = ComposableGraph.from_indices(
    ListIndex, 
    [index_set[ema_num] for ema_num in ema], 
    index_summaries=index_summaries,
    service_context=service_context
)

## Setting up the Chatbot Agent

We use Langchain to define the outer chatbot abstraction. We use LlamaIndex as a core Tool within this abstraction.

In [7]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from llama_index.langchain_helpers.agents import LlamaToolkit, create_llama_chat_agent, IndexToolConfig
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
from llama_index.query_engine.transform_query_engine import TransformQueryEngine

In [8]:
# define a decompose transform
decompose_transform = DecomposeQueryTransform(
    llm_predictor, verbose=True
)

# define custom query engines
custom_query_engines = {}
for index in index_set.values():
    query_engine = index.as_query_engine()
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_extra_info={'index_summary': index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine
custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    response_mode='tree_summarize',
    verbose=True,
)

# construct query engine
graph_query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)


In [9]:
index_configs = []

for y in range(1, 4):
    query_engine = index_set[y].as_query_engine(
        similarity_top_k=3,
    )
    tool_config = IndexToolConfig(
        query_engine=query_engine, 
        name=f"Vector Index {y}",
        description=f"Necessary for when you want to answer queries about solar energy, EMA's energy policy, and other energy policy related matters {y} ",
        tool_kwargs={"return_direct": True, "return_sources": True},
    )
    index_configs.append(tool_config)

graph_config = IndexToolConfig(
    query_engine=graph_query_engine,
    name=f"Graph Index",
    description="Necessary for when you want to answer queries regarding EMAs energy policy.",
    tool_kwargs={"return_direct": True, "return_sources": True},
    return_sources=True
)

toolkit = LlamaToolkit(
    index_configs=index_configs,
    graph_configs=[graph_config]
)

In [10]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm=OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
    verbose=True
)

In [11]:
prompt_inj = "Answer factually with only reference to the EMA documents."

# agent_chain.run(input="{prompt_inj} What is Singapore's goal for solar capacity by 2030?")
response = agent_chain.run(input="{prompt_inj} What is Singapore's goal for solar capacity by 2030?")




> Entering new  chain...

Thought: Do I need to use a tool? Yes
Action: Vector Index 3
Action Input: Singapore's goal for solar capacity by 2030
Observation: {'answer': "\nSingapore's goal for solar capacity by 2030 is to reach at least 2 gigawatt-peak.", 'sources': [{'start': 0, 'end': 1980, '_node_type': <NodeType.TEXT: '1'>, 'ref_doc_id': '89504346-2a6b-4edb-a1d6-a71e3d3dcd57', 'score': 0.9173613270297988}, {'start': 15617, 'end': 17654, '_node_type': <NodeType.TEXT: '1'>, 'ref_doc_id': '89504346-2a6b-4edb-a1d6-a71e3d3dcd57', 'score': 0.8995338012540366}, {'start': 1919, 'end': 3835, '_node_type': <NodeType.TEXT: '1'>, 'ref_doc_id': '89504346-2a6b-4edb-a1d6-a71e3d3dcd57', 'score': 0.89473478729985}]}


> Finished chain.


In [12]:
print(response)

{'answer': "\nSingapore's goal for solar capacity by 2030 is to reach at least 2 gigawatt-peak.", 'sources': [{'start': 0, 'end': 1980, '_node_type': <NodeType.TEXT: '1'>, 'ref_doc_id': '89504346-2a6b-4edb-a1d6-a71e3d3dcd57', 'score': 0.9173613270297988}, {'start': 15617, 'end': 17654, '_node_type': <NodeType.TEXT: '1'>, 'ref_doc_id': '89504346-2a6b-4edb-a1d6-a71e3d3dcd57', 'score': 0.8995338012540366}, {'start': 1919, 'end': 3835, '_node_type': <NodeType.TEXT: '1'>, 'ref_doc_id': '89504346-2a6b-4edb-a1d6-a71e3d3dcd57', 'score': 0.89473478729985}]}


In [ ]:
print(response)

### Setup Chatbot Loop Within Notebook

We'll keep a running loop so that you can converse with the agent. 

In [13]:
# reinitialize agent
memory = ConversationBufferMemory(memory_key="chat_history")
llm=OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
)

In [16]:
inj = """
        Your name is New_Jamie. 
        You are a Government Officer working for EMA in Singapore. You will answer only with reference to official documents from EMA. 
        Refer to the context FAQs and the EMA documents in composing your answers.
        If the user is unclear, you can ask the user to clarify the question.
        When in doubt,and/or the answer is not in the EMA documents, you can say "I am sorry but do not know the answer".
        Keep your answers short and as terse as possible. Be polite at all times.
    """

while True:
    text_input = input("User: ")
    response = agent_chain.run(input=text_input + inj)
    print(f'{text_input}')
    print(f'Agent: {response}')
    print(f'')


Hello
Agent: Hello, my name is New_Jamie and I am a Government Officer working for the Energy Market Authority in Singapore. How can I help you?

What's my name? 
Agent: Your name is New_Jamie.

What is your name? 
Agent: My name is New_Jamie.

How quaint that we both share the same name
Agent: Yes, it is quite interesting that we both share the same name.

What is Singapore's Hydrogen stretegy
Agent: Singapore's Hydrogen Strategy is part of the Energy Market Authority's Energy Reset pillar, which seeks to "transform Singapore's energy system to one that is more resilient, sustainable, and cost-effective". The strategy focuses on the development of a hydrogen economy in Singapore, which includes initiatives such as the deployment of hydrogen fuel cell vehicles, the development of hydrogen production and storage infrastructure, and the promotion of public awareness and acceptance of hydrogen as an energy source.


Agent: How can I help you?



KeyboardInterrupt: Interrupted by user